In [1]:
from utils.spark_config import get_spark_session
from configs.paths import ROOT_DIR

spark = get_spark_session()
spark

:: loading settings :: url = jar:file:/home/administrator/Desktop/datascience/github/learning-spark/.venv/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/administrator/.ivy2.5.2/cache
The jars for the packages stored in: /home/administrator/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a4733d95-fea6-4a8c-a836-3140170b15da;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
:: resolution report :: resolve 148ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-spark_2.13;4.0.0 from central in [default]
	io.delta#delta-storage;4.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.13.1 from central in [default]
	---------------------------------------------------------------------
	|             

In [2]:
bronze_path = f"{ROOT_DIR}/data/bronze/customer_churn_bronze"
features_path = f"{ROOT_DIR}/data/ml/customer_churn_features"

In [3]:
churn_data = spark.read.format("delta").load(bronze_path).select("CustomerID", "Churn")
churn_data.show()

25/08/21 11:29:18 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+-----+
|CustomerID|Churn|
+----------+-----+
|7590-VHVEG|   No|
|5575-GNVDE|   No|
|3668-QPYBK|  Yes|
|7795-CFOCW|   No|
|9237-HQITU|  Yes|
|9305-CDSKC|  Yes|
|1452-KIOVK|   No|
|6713-OKOMC|   No|
|7892-POOKP|  Yes|
|6388-TABGU|   No|
|9763-GRSKD|   No|
|7469-LKBCI|   No|
|8091-TTVAX|   No|
|0280-XJGEX|  Yes|
|5129-JLPIS|   No|
|3655-SNQYZ|   No|
|8191-XWSZG|   No|
|9959-WOFKT|   No|
|4190-MFLUW|  Yes|
|4183-MYFRB|   No|
+----------+-----+
only showing top 20 rows


In [4]:
features = spark.read.format("delta").load(features_path)
features.show()

+----------+-------------------------------------+-----------------------------+----------------------------+---------------------------------+---------------------------+---------------------------+----------------------------------------+------------------------------------+---------------------------------------------+-----------------------------------------------+-------------+-------+----------+------+------------+-------------+--------------+------------+----------------+-----------+-----------+---------------+----------------+--------------+------------+
|CustomerID|InternetService_index_ohe_Fiber_optic|InternetService_index_ohe_DSL|InternetService_index_ohe_No|Contract_index_ohe_Month_to_month|Contract_index_ohe_Two_year|Contract_index_ohe_One_year|PaymentMethod_index_ohe_Electronic_check|PaymentMethod_index_ohe_Mailed_check|PaymentMethod_index_ohe_Credit_card_automatic|PaymentMethod_index_ohe_Bank_transfer_automatic|SeniorCitizen|Partner|Dependents|tenure|PhoneService|Multipl

In [5]:
df = churn_data.join(features, on="CustomerID", how="inner")
df.show()

+----------+-----+-------------------------------------+-----------------------------+----------------------------+---------------------------------+---------------------------+---------------------------+----------------------------------------+------------------------------------+---------------------------------------------+-----------------------------------------------+-------------+-------+----------+------+------------+-------------+--------------+------------+----------------+-----------+-----------+---------------+----------------+--------------+------------+
|CustomerID|Churn|InternetService_index_ohe_Fiber_optic|InternetService_index_ohe_DSL|InternetService_index_ohe_No|Contract_index_ohe_Month_to_month|Contract_index_ohe_Two_year|Contract_index_ohe_One_year|PaymentMethod_index_ohe_Electronic_check|PaymentMethod_index_ohe_Mailed_check|PaymentMethod_index_ohe_Credit_card_automatic|PaymentMethod_index_ohe_Bank_transfer_automatic|SeniorCitizen|Partner|Dependents|tenure|PhoneSer

In [6]:
gold_path = f"{ROOT_DIR}/data/gold/customer_churn_gold"
df.write.format("delta").mode("overwrite").option("path", gold_path).saveAsTable("customer_churn_gold")

25/08/21 11:29:29 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
25/08/21 11:29:29 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore administrator@192.168.1.6
25/08/21 11:29:33 ERROR HiveAlterHandler: Failed to alter table default.customer_churn_gold
25/08/21 11:29:33 WARN HiveExternalCatalog: Could not alter schema of table `default`.`customer_churn_gold` in a Hive compatible way. Updating Hive metastore in Spark SQL specific format.
java.lang.reflect.InvocationTargetException
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:75)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:52)
	at java.base/java.lang.reflect.Method.in

In [7]:
spark.stop()